# Viewer Callbacks Example

Example using viewer callbacks to add markers to a recording.

In [ ]:
# serve the assets from the local install
from __future__ import annotations

%env RERUN_NOTEBOOK_ASSET=serve-local

In [ ]:
import io

import ipywidgets as widgets
import numpy as np
import rerun as rr
from IPython.display import display
from rerun.dataframe import Recording
from rerun.notebook import RecordingStream, SelectionItem, Viewer, ViewerCallbacks



In [ ]:
RAW_RECORDING_PATH = "/Users/gijsd/rerun/pusht_image_raw.rrd"
raw_recording = rr.dataframe.load_recording(RAW_RECORDING_PATH)
print("Loaded recording:", raw_recording.application_id(), raw_recording.recording_id())

annotated_recording = rr.new_recording(
    application_id=raw_recording.application_id(), recording_id=raw_recording.recording_id()
)
rr.log_file_from_path(file_path=RAW_RECORDING_PATH, recording=annotated_recording)

In [4]:
class RecordingMarker(ViewerCallbacks):
    def __init__(
        self,
        viewer: Viewer,
        annotated_recording: RecordingStream,
        raw_recording: Recording,
        default_entity_path: str | None = None,
    ) -> None:
        self.viewer = viewer
        self.annotated_recording = annotated_recording
        self.raw_recording = raw_recording
        self.timeline = "log_time"
        self.time = 0
        self.entity_path = default_entity_path if default_entity_path is not None else ""

        self.timestamp_button = widgets.Button(
            description="Add Timestamp", button_style="primary", tooltip="Mark current position in the recording"
        )

        self.comment_text = widgets.Text(
            placeholder="Optional comment", description="Comment:", style={"description_width": "initial"}
        )

        self.markers_output = widgets.Output(
            layout={
                "border": "1px solid #ddd",
                "padding": "10px",
                "margin": "10px 0px",
                "max_height": "300px",
                "overflow": "auto",
                "color": "white",
            }
        )

        self.time_label = widgets.Label(value=self._get_time_label_text())
        self.status_label = widgets.Label(value="")
        self.image_widget = widgets.Image(
            format="png",
            width=400,
            height=300,
            layout=widgets.Layout(border="1px solid #ddd", margin="10px 0px"),
        )

        self.timestamp_button.on_click(self.add_marker)
        self.widget = widgets.VBox(
            [
                self.time_label,
                widgets.HBox([self.timestamp_button, self.comment_text]),
                self.image_widget,
            ],
            layout=widgets.Layout(
                background_color="white",
            ),
        )

    def _get_time_label_text(self):
        """Helper to construct the text for the time label including entity path."""
        entity_info = f" | Selected: {self.entity_path}" if self.entity_path else ""
        return f"{self.timeline}: {self.time}{entity_info}"

    def _set_time(self, time: float, timeline: str | None = None):
        if timeline is None:
            timeline = self.timeline
        self.timeline = timeline

        # Check if time value can be safely converted to int64
        try:
            time = np.int64(time)
        except (OverflowError, ValueError):
            print(f"Time value {time} cannot be converted to int64, defaulting to 0")
            # if the time value is not a valid, default to 0
            time = np.int64(0)

        self.time = time
        self.update_timeline()

    def on_time_update(self, time: float) -> None:
        self._set_time(time)

    def on_timeline_change(self, timeline: str, time: float) -> None:
        self._set_time(time, timeline)

    def update_timeline(self) -> None:
        # Update the Label widget with time and entity path
        self.time_label.value = self._get_time_label_text()
        self.update_image()

    def on_selection_change(self, selection: list[SelectionItem]) -> None:
        for s in selection:
            if s.kind == "entity":
                self.entity_path = s.entity_path
                self.update_timeline()

    def add_marker(self, _button) -> None:
        """Add a new timestamp marker with optional comment."""
        comment = self.comment_text.value.strip()

        rr.set_time_sequence(timeline=self.timeline, sequence=self.time, recording=self.annotated_recording)
        rr.log(self.entity_path, rr.AnyValues(marker=(), comment=comment), recording=self.annotated_recording)

        self.status_label.value = f"✓ Marker added at {self.time} ({self.timeline})"
        self.comment_text.value = ""

    def update_image(self) -> None:
        if self.entity_path == "":
            return

        img = (
            raw_recording.view(index=self.timeline, contents=self.entity_path)
            .using_index_values(np.array([self.time]).astype(np.int64))
            .select(f"{self.entity_path}:Blob")
            .read_all()[f"{self.entity_path}:Blob"]
            .to_pandas()
        )

        if img.empty or img[0] is None:
            # no images found at this time, clear the image
            self.image_widget.value = io.BytesIO().getvalue()
            return

        # do some evil wrangling technique
        img = img[0][0]

        b = io.BytesIO(img.tobytes())
        b.seek(0)
        self.image_widget.value = b.getvalue()

    def display(self) -> None:
        """Display the widget."""
        display(self.widget)

In [ ]:
viewer = Viewer(
    width=960,
    height=720,
    recording=annotated_recording,
)
marker_callback = RecordingMarker(viewer, annotated_recording, raw_recording, default_entity_path="/observation.image")
viewer.register_callbacks(marker_callback)

display(viewer)
marker_callback.display()